# Tables Total Recall

The purpose of this notebook is to get to the point where we can extract 100% of part names from transistor hardware sheets.

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
load_pickle = False
save_pickle = False

corpus_loaded = False
if load_pickle:
    try:
        import cPickle
        with open("data/hardware/hardware_corpus.pkl","r") as pkl:
            %time corpus = cPickle.load(pkl)
        corpus_loaded = True
        print "Corpus has been loaded."
    except:
        print "Corpus could not be loaded."
        print "Corpus will be parsed instead..."
if not corpus_loaded:
    from snorkel.parser import CorpusParser
    from snorkel.parser import HTMLParser
    from snorkel.parser import TableParser

    doc_parser = HTMLParser(path='data/hardware/hardware_html/')
    context_parser = TableParser()

    cp = CorpusParser(doc_parser, context_parser, max_docs=15)
    %time corpus = cp.parse_corpus(name='Hardware Corpus')
    print "Corpus has been parsed."
    
    if save_pickle:
        with open("data/hardware/hardware_corpus.pkl","w") as pkl:
            %time cPickle.dump(corpus, pkl)
            print "Corpus has been pickled."

CPU times: user 15.2 s, sys: 671 ms, total: 15.9 s
Wall time: 22.3 s
Corpus has been parsed.


In [14]:
from load_dictionaries import load_hardware_dictionary

gold_parts = load_hardware_dictionary()
print "Loaded %s gold part numbers." % len(gold_parts)

Loaded 179 gold part numbers.


In [15]:
from snorkel.candidates import Ngrams, EntityExtractor
from snorkel.matchers import RegexMatchEach, DictionaryMatch, RangeMatcher

ngrams = Ngrams(n_max=2)
part_matcher = DictionaryMatch(d=gold_parts, longest_match_only=False)
part_extractor = EntityExtractor(ngrams, part_matcher)

%time parts = part_extractor.extract(corpus.get_sentences(), name='all')
for p in parts[:5]: 
    print p
print "Extracted %s candidate part numbers." % len(parts)

CPU times: user 302 ms, sys: 71.1 ms, total: 373 ms
Wall time: 322 ms
Span("BC547", context=None, chars=[0,4], words=[0,0])
Span("BC548", context=None, chars=[0,4], words=[0,0])
Span("BC547", context=None, chars=[0,4], words=[0,0])
Span("BC548", context=None, chars=[0,4], words=[0,0])
Span("BC546", context=None, chars=[0,4], words=[0,0])
Extracted 179 candidate part numbers.


In [16]:
from fractions import Fraction
g = set(gold_parts)
x = set([p.get_span() for p in parts])
recall = Fraction(len(g.intersection(x)),len(g))
precision = Fraction(len(g.intersection(x)),len(x))
print "Recall: %0.3f (%s/%s)" % (float(recall), recall.numerator, recall.denominator)
print "Precision: %0.3f (%s/%s)" % (float(precision), precision.numerator, precision.denominator)

Recall: 0.156 (28/179)
Precision: 1.000 (1/1)


In [17]:
print x - g
for p in x:
    print p

set([])
2N4123
2N4124
2N3906
PZT3906
BC547C
BC182B
BC549C
BC549B
BC550C
BC337
BC338
BC547A
MMBT3906
BC547B
BC549
BC548
BC547
BC546
MMBT4124
MMBT6427
BC550
BC548B
BC548C
BC546B
BC546C
BC546A
BC182
2N6426


In [ ]:
# ranges (e.g., BC546-BC548) or lists (e.g., BC546/547/548)